## Data preparement

In [26]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

"""Read dataframe"""
df1 = pd.read_csv("./ELECTROENCEPHALOGRAM_OF_SENSITIVITY_BODIES/specified_finger.csv",
                  delimiter=";", decimal=".", header=0)
df2 = pd.read_csv("./ELECTROENCEPHALOGRAM_OF_SENSITIVITY_BODIES/thumb_finger.csv",
                  delimiter=";", decimal=".", header=0)

"""Preprocessing data"""
#Addition of target variable
df1["target"] = "specified_finger"
df2["target"] = "thumb_finger"
#Concatenation dataframes

specified_finger = df1.sample(frac=0.1)
thumb_finger = df2.sample(frac=0.1)

print(specified_finger.shape)
print(thumb_finger.shape)

frames = [specified_finger, thumb_finger]

df = pd.concat(frames)

print(df.shape)

#Removing duplicate rows
df.drop_duplicates(subset=None, keep='first', inplace=True)
#Reset index of dataframe
df.reset_index(drop=True, inplace=True)
#Return column names of dataframe
column_names = list(df.columns)
#Checking data balance
target_var = df[column_names[-1]]
balance = Counter(target_var)
#Initialization of encoder
labelencoder = LabelEncoder()
#Encoding target variable
df["target"] = labelencoder.fit_transform(df["target"])

(8995, 17)
(9003, 17)
(17998, 17)


## Splitting to test & train

In [27]:
"""Splitting independent 'x' and dependent 'y'  variables of dataframe 'df'"""
x = df[column_names[0:len(column_names)-1]]
#Standardization or mean removal and variance scaling
x = (x-x.mean())/x.std()
#Converting to numpy array
x = x.values
#Create target variable
y = df[column_names[len(column_names)-1]].values

"""Splitting on training sample and test sample"""
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# MLP model

## MLP training

In [185]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation='logistic', batch_size=25, max_iter=200)

start_time = time.time()

mlp.fit(x_train, y_train)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

y_pred = mlp.predict(x_test)
y_pred_prob = mlp.predict_proba(x_test)

acc_score = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

acc_score, f1, roc_auc

Execution time: 35.71894598007202 seconds


C:\Users\Yuriy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


(0.7583333333333333, 0.7663802363050484, 0.7579698064277114)

# KAN model

## Data preparing

In [28]:
import torch

dtype = torch.get_default_dtype()
device = "cpu"

x_train = torch.from_numpy(x_train).type(dtype).to(device)
y_train = torch.from_numpy(y_train).type(dtype).to(device)

x_test = torch.from_numpy(x_test).type(dtype).to(device)
y_test = torch.from_numpy(y_test).type(dtype).to(device)

custom_dataset = {
    'train_input': x_train,
    'test_input': x_test,
    'train_label': y_train.view(-1, 1),
    'test_label': y_test.view(-1, 1)
}

## KAN training

In [253]:
from kan import KAN

model = KAN(width=[16, 3, 1], grid=3, k=3)

def train_acc():
    return torch.mean((torch.round(torch.sigmoid(model(custom_dataset['train_input']))[:,0]) == custom_dataset['train_label'][:,0]).type(dtype))

def test_acc():
    return torch.mean((torch.round(torch.sigmoid(model(custom_dataset['test_input']))[:,0]) == custom_dataset['test_label'][:,0]).type(dtype))

start_time = time.time()

print(custom_dataset['train_input'].dtype)
print(custom_dataset['test_input'].dtype)

results = model.fit(custom_dataset, opt="LBFGS", steps=100,
                    batch=x_test.shape[0], metrics=(train_acc, test_acc), loss_fn=torch.nn.BCEWithLogitsLoss())
end_time = time.time()

results['train_acc'][-1], results['test_acc'][-1]

checkpoint directory created: ./model
saving model version 0.0
torch.float32
torch.float32


| train_loss: 6.49e-01 | test_loss: 6.56e-01 | reg: 4.50e+01 | : 100%|█| 100/100 [01:01<00:00,  1.63

saving model version 0.1


(0.7744825482368469, 0.7799999713897705)

In [254]:
execution_time = end_time - start_time
execution_time

61.39775562286377

In [255]:
logits = model(x_test)
print(logits)
y_pred = torch.round(torch.sigmoid(logits))

tensor([[-3.6810],
        [-4.2083],
        [ 0.5502],
        ...,
        [ 0.8158],
        [14.8237],
        [-5.2607]], grad_fn=<AddBackward0>)


## KAN accuracy testing

In [256]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import numpy as np

y_test_numpy = y_test.detach().numpy()
y_pred_numpy = y_pred.detach().numpy()
#y_pred_numpy = np.where((y_pred_numpy < 0.5) | (y_pred_numpy < 0), 0, 1)

print(np.unique(y_pred_numpy))

#Compute the 'accuracy-score'
acc_score = accuracy_score(y_test_numpy, y_pred_numpy)
#Compute the 'f1-score'
f1 = f1_score(y_test_numpy, y_pred_numpy)
#Compute the 'roc-auc'
roc_auc = roc_auc_score(y_test_numpy, y_pred_numpy)

acc_score, f1, roc_auc

[0. 1.]


(0.78, 0.7891373801916933, 0.779494303973326)